<a href="https://colab.research.google.com/github/kenny08gt/lab1_algoritmos_ds/blob/master/lab1_algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Alan Hrtarte
# 19000590

---

# Laboratorio 1

---

In [0]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

# Método de Newton

In [0]:
def eqFromStr(str):
  str = str.strip()
  str = str.lower()
  arr = re.split("([\+|\-])", str)
  out = ""
  for el in arr:
    el = el.replace('^', '**')
    if 'x' in el:
      var_pos = el.index('x')
      el = '(' + el[:var_pos] + '*' + el[var_pos:] + ')'
     
    out = out + el
  return out

def f(eq, x):
  strOut = eqFromStr(eq)
  return eval(strOut)

def derivative(str_equ, x, h):
  func = eqFromStr(str_equ)
  func = func.replace("x", '(x + h)')
  func = "-4*(" + func + ")"
  out = eval(func)
  
  func = eqFromStr(str_equ)
  func = func.replace("x", '(x + 2*h)')
  out = out + eval(func)
  
  func = eqFromStr(str_equ)
  func = func.replace("x", '(x)')
  func = "3*(" + func + ")"
  out = out + eval(func)
  
  out = -out/(2*h)
  
  return out

def Newton(eqStr, x0, epsilon, h):
  
  xn = x0
  error = 1
  epochs = []
  f_vals = []
  devs = []
  Xn = []
  errors = []
  i = 0
  while(error > epsilon):
    f_val = f(eqStr, xn)
    dev = derivative(eqStr, xn, h)
    x_n1 = xn - f_val/dev
    error = abs(f(eqStr, xn))
    i = i + 1
    xn = x_n1
    epochs.append(i)
    Xn.append(xn)
    errors.append(error)
    f_vals.append(f_val)
    devs.append(dev)
  
  results = pd.DataFrame({'Iter':epochs, 'Xn':Xn, 'Error': errors, 'F(x)':f_vals, "f'(x)":devs})
  return results, xn
  
  
data, solution = Newton('4x^4 - 4x + 1 ', 0.1, 0.0000001, 0.000001)
data

,Iter,Xn,Error,F(x),f'(x)
0,1,0.250703,6.004000e-01,6.004000e-01,-3.984000
1,2,0.254169,1.299020e-02,1.299020e-02,-3.747886
2,3,0.254174,1.828887e-05,1.828887e-05,-3.737284
3,4,0.254174,3.712897e-11,3.712897e-11,-3.737269


In [0]:
4x
4 − 4x + 1

In [0]:
def estimateRC(Xn, sol):
  error = []
  for xn in Xn:
    error.append(abs(xn - sol))

  ek1 = error[0:len(error)-3]
  ek = error[1:len(error)-2]
  
  y = np.array(ek)
  x = np.array(ek1)
  x = np.log(x)
  y = np.log(y)
  
  x = x.reshape(-1, 1)
  y = y.reshape(-1, 1)

  model = LinearRegression()  
  model.fit(x,y) 
  r = model.coef_
  C =  model.intercept_
  C = np.exp(C)
  return C,r

In [0]:
estimateRC(data['Xn'], solution)

(array([4.89363574e-06]), array([[0.]]))

## Conclusión newton
La solución que encontro el método de newton es un mínimo global ya que la Hessian es PD.
C = 4.8936e-06 y r = 0

# Método de Bisección

In [0]:
def bisection(f_str, a, b, distance):
  error = 1
  epochs = []
  f_vals = []
  devs = []
  Xn = []
  errors = []
  i = 0
  
  fa = f(f_str, a)
  fb = f(f_str, b)
  
  if fa * fb >= 0:
    return None
  a_n = a
  b_n = b
  
  while ((b_n-a_n) >= distance):
    m_n = (a_n + b_n)/2
    f_m_n = f(f_str, m_n)
    f_a_n = f(f_str, a_n)
    f_b_n = f(f_str, b_n)
    if f_a_n * f_m_n < 0:
      a_n = a_n
      b_n = m_n
    elif f_b_n * f_m_n < 0:
      a_n = m_n
      b_n = b_n
    elif f_m_n == 0:
      return m_n
    else:
      return None
    
    error = abs(f(f_str, m_n))
    epochs.append(i)
    Xn.append(m_n)
    f_vals.append(f_m_n)
    errors.append(error)
    i=i+1
  data = pd.DataFrame({'Iter':epochs, 'Xn':Xn, 'Error': errors, 'F(x)':f_vals })
  return data, m_n

In [0]:
data, sol = bisection('4x^4 - 4x + 1 ', 0, 0.5, 0.001)
data

,Iter,Xn,Error,F(x)
0,0,0.250000,0.015625,0.015625
1,1,0.375000,0.420898,-0.420898
2,2,0.312500,0.211853,-0.211853
3,3,0.281250,0.099972,-0.099972
4,4,0.265625,0.042587,-0.042587
5,5,0.257812,0.013578,-0.013578
6,6,0.253906,0.001000,0.001000
7,7,0.255859,0.006295,-0.006295
8,8,0.254883,0.002649,-0.002649


In [0]:
estimateRC(data['Xn'], sol)

(array([0.1474194]), array([[0.59347774]]))

# Newton vs Bisección
* El método de bisección requiere conocimiento de la forma la función para hacer una buena estimación de los puntos iniciales (a, b)
* Newton requiere mas computo, por el calculo de la derivada. Sin embargo converge maas rapido que el de bisección.